<a href="https://colab.research.google.com/github/htbkoo/0Colaboratory/blob/master/Kaggle/titanic-machine-learning-from-disaster/(xgboost)_htbkoo_Titanic__Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

# Any results you write to the current directory are saved as output.

/home/hey/.pyenv/versions/3.8.5/envs/self_jupyter_notebooks/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
data_folder = './local_data'
print(os.listdir(data_folder))

['gender_submission.csv', 'test.csv', 'train.csv']


In [3]:
train_data_path = f'{data_folder}/train.csv'
test_data_path = f'{data_folder}/test.csv'

train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

In [4]:
train_data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [7]:
columns = test_data.columns
columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [8]:
train_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [9]:
ALL_FEATURES = ['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
ALL_FEATURES

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [10]:
# features = [ 'Pclass', 'Name', 'Sex', 'Age', 'SibSp','Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
features = [ 'Pclass', 'Sex', 'Age', 'SibSp','Parch', 'Fare', 'Embarked']
features

['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']

In [11]:
train_X = train_data[features]
train_X.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [15]:
# [col for col in train_data.columns if train_data[col].isnull().any()]

len(train_data[train_data['Age'].isnull()])

# len(train_data)

177

In [25]:
[col for col in train_data.columns if train_data[col].isnull().any()]

len(train_data[train_data['Age'].isnull()])

len(train_data)

891

In [ ]:
missing_val_count_by_column = (train_X.isnull().sum())
missing_val_count_by_column
# print(missing_val_count_by_column[missing_val_count_by_column > 0

In [ ]:
train_X["Sex"] =  [0 if ((sex == 'female') or (sex == 0)) else 1 for sex in train_X["Sex"]]
train_X["Embarked"] =  [0 if ((embarked == 'C') or (embarked == 0)) else 1 if ((embarked == 'Q') or (embarked == 1)) else 2 for embarked in train_X["Embarked"]]
train_X.head()

In [ ]:
age_mean = train_X.Age.mean()
age_mean

In [ ]:
train_X = train_X.fillna({'Age': age_mean, 'Embarked': 0})

In [ ]:
missing_val_count_by_column = (train_X.isnull().sum())
missing_val_count_by_column

In [ ]:
train_y = train_data.Survived
train_y.head()

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor(random_state=1)

In [ ]:
rf_model.fit(train_X, train_y)

In [ ]:
test_X = test_data[features]
test_X.head()

In [ ]:
missing_val_count_by_column = (test_X.isnull().sum())
missing_val_count_by_column

In [ ]:
test_X["Sex"] =  [0 if ((sex == 'female') or (sex == 0)) else 1 for sex in test_X["Sex"]]
test_X["Embarked"] =  [0 if ((embarked == 'C') or (embarked == 0)) else 1 if ((embarked == 'Q') or (embarked == 1)) else 2 for embarked in test_X["Embarked"]]
test_X.head()

In [ ]:
test_age_mean = test_X.Age.mean()
test_fare_mean = test_X.Fare.mean()

In [ ]:
test_X = test_X.fillna({'Age': test_age_mean, 'Fare': test_fare_mean})

In [ ]:
missing_val_count_by_column = (test_X.isnull().sum())
missing_val_count_by_column

In [ ]:
test_preds = rf_model.predict(test_X)
test_preds[:10]

In [ ]:
threshold = 0.5
test_preds = [0 if pred<threshold else 1 for pred in test_preds]
test_preds[:10]

In [ ]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId,
                      'Survived': test_preds})
output.to_csv('submission.csv', index=False)

In [ ]:
[col for col in [1,2,3,4] if col!=1]

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

# read in data
dtrain = xgb.DMatrix(trains_X.values, trains_Y.values)
dtest = xgb.DMatrix(test_X.values)
# specify parameters via map
param = {'max_depth':2, 'eta':1, 'objective':'binary:logistic' }
num_round = 2
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtest)

mean_absolute_error(preds, )